# Managing the Db2 Data Management Console

This Jupyter Notebook contains examples of how to setup and manage the Db2 Data Management Console. It covers how to add additional users using database authentication, how to explore and manage connections and setup and manage monitoring profiles.

The Db2 Data Management Console is more than a graphical user interface. It is a set of microservices that you can use to build custom applications to automate your use of Db2.

This Jupyter Notebook contains examples of how to use the Open APIs and the composable interface that are available in the Db2 Data Management Console. Everything in the User Interface is also available through an open and fully documented RESTful Services API. The full set of APIs are documented as part of the Db2 Data Management Console user interface. In this hands on lab you can connect to the documentation directly through this link: [Db2 Data Management Console RESTful APIs](http://localhost:11080/dbapi/api/index_enterprise.html). 

You can also embed elements of the full user interface into an IFrame by constructing the appropriate URL.

This hands on lab will be calling the Db2 Data Management Console as a service. However you can explore it through the user interface as well. Just click on the following link to try out the console that is already and setup in this lab: http://localhost:11080/console. If you have not already logged in you can use the following:
* Userid: db2inst1
* Password: db2inst1

### Import Helper Classes
For more information on these classes, see the Lab on Automate Db2 with Open Console Services

### The Console Class
Next we will create a Console helper class that will encapsulate the Rest API calls that we can use to directly access the Db2 Data Management Console service without having to use the user interface. 

To access the service we need to first authenticate with the service and create a reusable token that we can use for each call to the service. This ensures that we don't have to provide a userID and password each time we run a command. The token makes sure this is secure. 

Each request is constructed of several parts. First, the URL and the API identify how to connect to the service. Second the REST service request that identifies the request and the options. For example '/metrics/applications/connections/current/list'. And finally some complex requests also include a JSON payload. For example running SQL includes a JSON object that identifies the script, statement delimiters, the maximum number of rows in the results set as well as what do if a statement fails.

The full set of APIs are documents as part of the Db2 Data Management Console user interface. In this hands on lab you can connect to that directly through this link: [Db2 Data Management Console RESTful APIs](http://localhost:11080/dbapi/api/index_enterprise.html). 

### First we will import a few helper classes
We need to pull in a few standard Python libraries so that we can work with REST, JSON and a library called Pandas. Pandas lets us work with DataFrames, which are a very powerful way to work with tabular data in Python. 

In [1]:
# Import the class libraries 
import requests
import ssl
import json
from pprint import pprint
from requests import Response
import pandas as pd
import time
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
from IPython.display import IFrame
from IPython.display import display, HTML
from pandas.io.json import json_normalize
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [2]:
# Run the Db2 Class library
# Used to construct and reuse an Autentication Key
# Used to construct RESTAPI URLs and JSON payloads
class Db2Console():
    
    def __init__(self, url, verify = False, proxies=None, ):
        self.url = url
        self.proxies = proxies
        self.verify = verify

    def authenticate(self, userid, password, profile=""):
        credentials = {'userid':userid, 'password':password}
        r = requests.post(self.url+'/auth/tokens', verify=self.verify, json=credentials, proxies=self.proxies)
        if (r.status_code == 200):
            bearerToken = r.json()['token']
            if profile == "":
                self.headers = {'Authorization': 'Bearer'+ ' '+bearerToken}
                return True;
            else:
                self.headers = {'Authorization': 'Bearer'+ ' '+bearerToken, 'X-DB-Profile': profile}
                return True;
        else:
            print ('Unable to authenticate, no bearer token obtained')
            return False;
        
    def printResponse(self, r, code):
        if (r.status_code == code):
            pprint(r.json())
        else:
            print (r.status_code)
            print (r.content)
    
    def getRequest(self, api, json=None):
        return requests.get(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json)

    def postRequest(self, api, json=None):
        return requests.post(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json) 
        
    def getStatusCode(self, response):
        return (response.status_code)

    def getJSON(self, response):
        return (response.json())
    
    def runSQL(self, script, limit=10, separator=';', stopOnError=False):
        sqlJob = {'commands': script, 'limit':limit, 'separator':separator, 'stop_on_error':str(stopOnError)}
        return self.postRequest('/sql_jobs',sqlJob)
        
    def getSQLJobResult(self, jobid):
        return self.getRequest('/sql_jobs/'+jobid)
    
    def getUserPriviledges(self):
        return self.getRequest('/userProfilePrivileges')
    
    def getProfile(self,profile):
        return self.getRequest('/dbprofiles/'+profile)    
    
    def getMonitorStatus(self):
        return self.getRequest('/monitor') 
    
    def getConsoleProfiles(self):
        return self.getRequest('/dbprofiles')  
    
    def getConsoleRepository(self):
        return self.getRequest('/repository')    

### Db2 Data Management Console Connection
To connect to the Db2 Data Management Console service you need to provide the URL, the service name (v4) and profile the console user name and password as well as the name of the connection profile used in the console to connect to the database you want to work with. For this lab we are assuming that the following values are used for the connection:
* Userid: db2inst1
* Password: db2inst1
* Connection: sample

**Note:** If the Db2 Data Management Console has not completed initialization, the connection below will fail. Wait for a few moments and then try it again.

In [3]:
# Connect to the Db2 Data Management Console service
Console  = 'http://localhost:11080'
user     = 'DB2INST1'
password = 'db2inst1'

# Set up the required connection
databaseAPI = Db2Console(Console+'/dbapi/v4')
if databaseAPI.authenticate(user, password) :
    print("Token Created")
else : 
    print("Token Creation Failed")
database = Console

Token Created


### Confirm the connection
To confirm that your connection is working you can the Console connection profiles.

In [4]:
# Get Console Connection Profiles
r = databaseAPI.getConsoleProfiles()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    display(pd.DataFrame(json_normalize(json)))
else:
    print(databaseAPI.getStatusCode(r))      

,name,disableDataCollection,databaseVersion,databaseName,timeZone,databaseVersion_VRMF,sslConnection,userProfileRole,host,_PROFILE_INIT_,dataServerType,port,edition,dataServerExternalType,capabilities,location,tag
0,SAMPLE,false,11.5.0,SAMPLE,-50000,11.5.0.0,false,OWNER,localhost,true,DB2LUW,50000,"AESE,DEC",DB2LUW,"[""DSM_ENTERPRISE_LUW""]",,[HOT]
1,HISTORY,false,11.5.0,HISTORY,-50000,11.5.0.0,false,OWNER,localhost,true,DB2LUW,50000,"AESE,DEC",DB2LUW,"[""DSM_ENTERPRISE_LUW""]",,NaN


### Get Users
You can also get a list of current users.

In [5]:
# Get Console Connection Profiles
r = databaseAPI.getUserPriviledges()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    display(pd.DataFrame(json_normalize(json)))
else:
    print(databaseAPI.getStatusCode(r))      

,profileName,OWNER,USER
0,SAMPLE,[db2inst1],"[db2pot, dmcusr]"
1,HISTORY,[db2inst1],"[db2pot, dmcusr]"


### Get Repository Configuration
You can also get details on the repository configuration

In [11]:
# Get Console Repository Configuration
r = databaseAPI.getConsoleRepository()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r) 
    display(pd.DataFrame(json_normalize(json)[['databaseName','status','host','port','collectionCred.user']]).transpose())
else:
    print(databaseAPI.getStatusCode(r))      

,0
databaseName,history
status,active
host,localhost
port,50000
collectionCred.user,db2inst1


#### Credits: IBM 2019, Peter Kohlmann [kohlmann@ca.ibm.com]